# Chapter 2 – End-to-end Machine Learning project

In [1]:
import numpy as np
import pandas as pd

In [2]:
import pickle

with open('datasets/housing_train_prepared.pickle', 'rb') as f:
    housing_train_data = pickle.load(f)
with open('datasets/housing_train_labels.pickle', 'rb') as f2:
    housing_train_labels_pandas = pickle.load(f2)
# Testen, ob alles gut gegangen ist
housing_train_data[:10]
housing_train_labels =housing_train_labels_pandas.to_numpy()
housing_train_labels
# Umwandlung in numpy hätte eigentlich schon in Teil 2 des Projekts stattfinden sollen

array([153000., 236700., 143300., ..., 278100., 500001.,  90600.])

# Training und Bewertung von Modellen


###### Schritte während des Lern-Prozesses

In [3]:
"""
0.  Algorithmus importieren und eine Instanz anlegen

1.  Algorithmus trainieren.
    Die Funktion dazu heißt immer "fit".
    Während des Trainings erstellt der Algorithmus das Modell, 
    von dem er glaubt, dass es die Daten gut beschreibt.
    
2.  Wenn das Modell fertig ist, versucht der Mensch abzuschätzen , 
wie gut es ist
    Für Voraussagen nutzen wir die Funktion predict , 
    die jeder Algo hat
    
    a) Wir machen Voraussagen auf Beispieldaten und schätzen ab, ob das echte Label getroffen wurde.
    b) Wir berechnen eine Kennzahl für die Voraussagen .
       Hiermit wird die Qualität von allen Voraussagen  zusammengefasst.
       Grundlage ist die TRAININGS-MENGE
       
       Für die Bewertung nutzen wir Funktionen, 
           z.B. mean_absolute_error,...
       
3.  Kreuzvalidierung mit cross_val_score
        Aufteilung in n Folds
        Ergebnis: n Scores
        Zusammenfassung der n Scores: z.B Mittelwert + Standardabweichung
4.  GridSearch: suche die Parameter, 
        die einen guten Algorithmus noch verbessern
        
        berechnet den Score für jede Parameterkombination über crossval
        Schleife: 
        pro Parameterkombination und Fold ein Training und eine Bewertung
        
5.  Voting (Klasse: VotingRegressor importieren und anwenden)

6.  Bewertung des endgültigen Modells durch Berechnung einer Kennzahl. 
    Diesmal wird die Kennzahl auf der TEST-MENGE berechnet
"""
print()

###### 1__LinReg) Training mit LinearRegression

In [4]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_train_data, housing_train_labels)

LinearRegression()

###### 2a-__LinReg) Stichprobe anschauen und die Leistung des Algorithmus überprüfen

In [5]:
# Wir prüfen an 5 Original-Zeilen (den "Beispielen") , ob der Algorithmus imstande ist, 
# die Label einigermaßen richtig vorauszusagen.
# Die Originalzeilen müssen mit der Pipeline vorbereitet werden

In [6]:
#beispiele_vorbereitet=pipe.transform(beispiele_data)
np.set_printoptions(precision=2)
beispiele_vorbereitet = housing_train_data[:5]
beispiele_labels = housing_train_labels[:5]
beispiele_vorbereitet


array([[-7.20e-02, -1.11e-01,  2.20e-01,  2.64e-01, -1.05e+00, -5.80e-04,
         1.49e-01, -3.86e-01,  8.53e-01,  2.92e+00, -5.03e-01,  1.00e+00,
         0.00e+00,  0.00e+00,  0.00e+00,  0.00e+00],
       [-3.41e-01, -3.01e-01, -8.00e-01,  2.33e-01, -5.57e-01,  3.18e-02,
        -6.14e-01,  5.99e-01, -8.65e-01,  3.40e-01, -4.74e-01,  1.00e+00,
         0.00e+00,  0.00e+00,  0.00e+00,  0.00e+00],
       [-6.99e-01,  7.55e-01, -6.03e-01, -3.36e-01,  3.84e-01,  1.02e-01,
        -5.55e-01, -2.15e-01, -2.94e-02,  7.55e-02, -6.85e-02,  1.00e+00,
         0.00e+00,  0.00e+00,  0.00e+00,  0.00e+00],
       [ 2.01e-01,  6.16e+00, -3.18e+00, -2.76e+00, -1.75e+00, -7.34e-01,
         2.52e+00, -1.04e-01, -7.71e-01, -3.34e-01, -6.27e-02,  0.00e+00,
         1.00e+00,  0.00e+00,  0.00e+00,  0.00e+00],
       [-2.38e-01,  3.59e-01, -1.08e+00, -2.36e-01,  3.52e-01,  2.10e-03,
        -7.78e-01,  4.47e-01,  2.77e-02,  1.84e-01, -6.80e-01,  1.00e+00,
         0.00e+00,  0.00e+00,  0.00e+00,  0.00e+

In [7]:
beispiele_labels

array([153000., 236700., 143300., 310000., 169000.])

In [8]:
beispiele_predictions = lin_reg.predict(beispiele_vorbereitet)
np.set_printoptions(precision=0)
beispiele_predictions
#lin_reg.coef_

array([192165., 303954., 187927., -92588., 224119.])

In [9]:
lin_reg.coef_

array([ 12594., -35978.,  -4503.,  20080.,  14359.,   3013., -10570.,
        23508., -63429.,  16752.,   5507., -39424., -76884., 195009.,
       -43010., -35691.])

In [8]:
# Vergleichen mit den echten Werten

In [11]:
beispiele_labels

array([153000., 236700., 143300., 310000., 169000.])

In [12]:
#Wie gut sind die Voraussagen?
#Als Antwort berechnen wir den prozentualen Fehler für jeden einzelnen Datensatz.
prozentuale_abweichungen = beispiele_predictions / beispiele_labels - 1
print(", ".join([f"{100 * ratio:.1f}%" for ratio in prozentuale_abweichungen]))

25.6%, 28.4%, 31.1%, -129.9%, 32.6%


######  2b__LinReg) Bewertung anhand von Kennzahlen, die aus dem gesamten Trainingsergebnis abgeleitet werden

In [18]:
housing_predictions_lin = lin_reg.predict(housing_train_data)
housing_predictions_lin[:5]
housing_train_labels[:5]

array([153000., 236700., 143300., 310000., 169000.])

In [19]:
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(housing_train_labels, housing_predictions_lin )
print("Durchschnittliche Abweichung zwischen echt und vorausgesagt(MAE):", round(lin_mae),"$")

Durchschnittliche Abweichung zwischen echt und vorausgesagt(MAE): 49369 $


In [20]:
from sklearn.metrics import mean_squared_error

lin_rmse = mean_squared_error(housing_train_labels, housing_predictions_lin, squared=False)
print("Durchschnittliche Abweichung zwischen echt und vorausgesagt(RMSE):", round(lin_rmse),"$")

Durchschnittliche Abweichung zwischen echt und vorausgesagt(RMSE): 67827 $


In [21]:
from sklearn.metrics import mean_absolute_percentage_error
lin_mape = mean_absolute_percentage_error(
                housing_train_labels, 
                housing_predictions_lin)
print("Prozentualer Fehler",np.round(lin_mape *100),"%")

Prozentualer Fehler 29.0 %


In [15]:
# Wir probieren den nächsten Algorithmus

###### 1__Tree) Training mit DecisionTree

In [22]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_train_data, housing_train_labels)

DecisionTreeRegressor(random_state=42)

######  2b__Tre)  Bewertung anhand von Kennzahlen, die aus dem gesamten Trainingsergebnis abgeleitet werden

In [23]:
housing_predictions_tree = tree_reg.predict(housing_train_data)
housing_predictions_tree[:6]

array([153000., 236700., 143300., 310000., 169000., 112500.])

In [24]:

tree_mae = mean_absolute_error(housing_train_labels, housing_predictions_tree)
tree_mae

0.0

In [25]:
housing_predictions = tree_reg.predict(housing_train_data)
tree_rmse = mean_squared_error(housing_train_labels, housing_predictions, squared=False)
tree_rmse

0.0

In [20]:
# Haben wir nun den perfekten Algorithmus gefunden ???

## 3. Bessere Bewertungsgrundlage: Cross-Validation (Kreuz-Evaluation)

In [21]:
# Der Trainingsscore  des DecisionTrees ist deutlich besser als der Crossvalscore.
# Das nennt man OVERFITTING .
# Das muss geändert werden, ehe man dem Score vertraut. 
# Wenn das nicht zu ändern ist, taugt der Algorithmus nichts für diese Daten.

In [26]:
from sklearn.model_selection import cross_val_score,cross_validate

### 3 Scoring-Verfahren für den DecisionTree

#### DecisionTree ________Scoring-Verfahren : root mean squared error

In [27]:
tree_rmses = -cross_val_score(tree_reg, housing_train_data, housing_train_labels,
                              scoring="neg_root_mean_squared_error", cv=10)
print("Evaluierungs-Scores DecisionTree, Verfahren RMSE")
tree_rmses
#crossval_score(instanz_des_algorithmus, 
#               trainingsdaten,trainingslabel, 
#               scoring=Bewertungsmaßstab,
#               cv = Anzahl_der_folds)

Evaluierungs-Scores DecisionTree, Verfahren RMSE


array([80260., 77447., 79196., 78331., 79101., 78090., 78814., 80230.,
       79826., 76461.])

In [28]:
print("Mittelwert aller Teil-RMSE-Scores (DecisionTree):",round(tree_rmses.mean()),
      "_____ Standardabweichung:",round(tree_rmses.std()))
#pd.DataFrame(tree_rmses).describe()

Mittelwert aller Teil-RMSE-Scores (DecisionTree): 78776 _____ Standardabweichung: 1162


#### DecisionTree__________Scoring-Verfahren :Mean absolute Error

In [29]:
tree_maes = -cross_val_score(tree_reg, housing_train_data, housing_train_labels,
                              scoring="neg_mean_absolute_error", cv=10)
tree_maes

array([53898., 52577., 53089., 51629., 53072., 52008., 52309., 52988.,
       52676., 51953.])

In [30]:
print("Mittelwert aller Teil-MAE-Scores (DecisionTree):",round(tree_maes.mean()),
      "$_____ Standardabweichung der Fehler:",round(tree_maes.std()),"$")
#pd.DataFrame(tree_rmses).describe()

Mittelwert aller Teil-MAE-Scores (DecisionTree): 52620 $_____ Standardabweichung der Fehler: 640 $


#### DecisionTree_________Scoring-Verfahren: Mean absolute percentage error

In [31]:
tree_mapes = -cross_val_score(tree_reg, housing_train_data, housing_train_labels,
                              scoring="neg_mean_absolute_percentage_error", cv=10)
np.round(tree_mapes *100)

array([29., 30., 29., 28., 29., 27., 28., 29., 29., 29.])

In [34]:
print("Mittelwert aller Teil-MAPE-Scores (DecisionTree):",round(tree_mapes.mean()*100),
      "%_____ Standardabweichung der Fehler:",round(tree_mapes.std()*100,2),"%")
#pd.DataFrame(tree_rmses).describe()

Mittelwert aller Teil-MAPE-Scores (DecisionTree): 29 %_____ Standardabweichung der Fehler: 0.73 %


### Zum Vergleich: Die Scores der linearen Regression

##### Lineare Regression____ Scoring

In [39]:
lin_mape_cv = -cross_val_score(lin_reg, housing_train_data, housing_train_labels,
                              scoring="neg_mean_absolute_percentage_error", cv=10)
print("Mittelwert aller Teil-MAPE-Scores  (LineareRegression):",round(lin_mape_cv.mean()*100),
      "%_____ Standardabweichung:",round(lin_mape_cv.std()*100,2),"%")

Mittelwert aller Teil-MAPE-Scores  (LineareRegression): 29 %_____ Standardabweichung: 1.02 %


In [40]:
lin_rmses = -cross_val_score(lin_reg, housing_train_data, housing_train_labels,
                              scoring="neg_root_mean_squared_error", cv=10)
print(f"Mittelwert aller Teil-RMSE-Scores  (LineareRegression): {round(lin_rmses.mean())} $",
      f"_____ Standardabweichung: {round(lin_rmses.std())} $")
#pd.Series(lin_rmses).describe()


Mittelwert aller Teil-RMSE-Scores  (LineareRegression): 68675 $ _____ Standardabweichung: 3092 $


In [34]:
# Der Trainingsscore der Linearen Regression stimmt zwar ungefähr mit der Crossval-Score über ein.
# Aber der Trainings-Score ist schlecht.
# Das nennt man UNDER-FITTING.
# Wir vertrauen diesem Algorithmus ebenfalls nicht.

### Nächster Versuch: Wir trainieren und evaluieren den Algorithmus RandomForest.
**Warning:** the following cell may take a few minutes to run:

In [41]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state=42, n_jobs = -1)
forest_rmses = -cross_val_score(forest_reg, housing_train_data, housing_train_labels,
                                scoring="neg_root_mean_squared_error", cv=5)

In [42]:
print(f"Durchschnitt der CV-Scores(RMSE): {round(forest_rmses.mean())} $", 
      f"_____ Standardabweichung: {round(forest_rmses.std())} $")
#pd.Series(forest_rmses).describe()

Durchschnitt der CV-Scores(RMSE): 55788 $ _____ Standardabweichung: 1069 $


In [43]:
forest_mapes = -cross_val_score(forest_reg, housing_train_data, housing_train_labels,
                                scoring="neg_mean_absolute_percentage_error", cv=5)
forest_mapes *100

array([22., 21., 21., 21., 21.])

In [44]:
print(f"Durchschnitt der CV-Scores(MAPE): {round(forest_mapes.mean()*100)} %",
      f"_____ Standardabweichung: {round(forest_mapes.std()*100)} %")
#(pd.Series(forest_mapes)*100).describe()

Durchschnitt der CV-Scores(MAPE): 21 % _____ Standardabweichung: 1 %


In [39]:
# Dieser Algorithmus liefert schon bessere Werte.
# Wir versuchen demnächst, die Bewertung weiter zu verbessern, indem wir nicht mehr mit den Default-Parametern arbeiten.
# Wir wollen die besten Varianten für die Parameter herausfinden.

### Nächster Versuch: Wir trainieren und evaluieren den Algorithmus "Support Vector Machine".

Warnung: die Programmausführung dauert ziemlich lang.

In [40]:
"""
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR

svr = SVR()
svr.fit(housing_train_data, housing_train_labels)
svr_train_predictions = svr.predict(housing_train_data)
print(round(
    mean_squared_error(
        housing_train_labels,svr_train_predictions,squared=False)))

#svr_rmses = -cross_val_score(svr, housing_train_data, housing_train_labels,
#                               scoring="neg_root_mean_squared_error", cv=5)
# Ergebnis rmse =111_000 $
"""
print()

# Fine-Tuning: Die Modellparameter werden optimiert

## Grid Search mit RMSE

Warnung: die Programmausführung  eines Gridsearch dauert sehr lang.

In [45]:

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = [
    { 'max_features': [2,4,6,8],
      'n_estimators': [3,10,30]}# oder ...[10,100,1000,10000]
#,
#    {'bootstrap': [False],
#     'max_features': [2, 3,4],
#     'n_estimators': [3, 10]}
]
#param_grid enthält eine Dictionary, das ausgewählte Parameter von der Klasse RandomForest zeigt.
# Diese Parameter heißen HYPER-PARAMETER
# Die Listen, die hinter den Parametern stehen , enthalten mögliche Werte der Parameter.

grid_search = GridSearchCV(RandomForestRegressor(random_state=42), 
                           param_grid, 
                           cv=3,
                           scoring='neg_root_mean_squared_error',
                           n_jobs = -1)
grid_search.fit(housing_train_data, housing_train_labels)
# Das grid_search-Objekt führt ein fit für jede Parameter-kombination aus und macht auch eine Kreuz-Evaluierung(Crossval)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]}],
             scoring='neg_root_mean_squared_error')

In [42]:
# Anmerkung:
# Man kann auch einen Gridsearch über alle Parameter einer Pipeline machen.
# Die Liste der Parameter kann man herausfinden mit :`full_pipeline.get_params().keys()`

In [46]:
# Die grid_search-Instanz findet die besten Parameter
grid_search.best_params_
# Anmerkung : der Score wird hier sehr schlecht, weil die Parameter im Gridsearch ungünstig gewählt waren.

{'max_features': 8, 'n_estimators': 30}

In [47]:
# Die grid_search-Instanz kann auch einen Estimator erstellen, der die besten Parameter schon enthält.
final_model=grid_search.best_estimator_
final_model

RandomForestRegressor(max_features=8, n_estimators=30, random_state=42)

Let's look at the score of each hyperparameter combination tested during the grid search:

In [48]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False,
                   inplace=True)
cv_res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
11,7.404942,0.057018,0.036745,1.185818e-03,8,30,"{'max_features': 8, 'n_estimators': 30}",-56307.759179,-55902.501324,-57429.478981,-56546.579828,645.854301,1
8,6.890174,0.036886,0.039930,1.581218e-04,6,30,"{'max_features': 6, 'n_estimators': 30}",-56641.604309,-56607.720035,-57791.547673,-57013.624006,550.249011,2
5,5.483845,0.113311,0.054363,2.820580e-03,4,30,"{'max_features': 4, 'n_estimators': 30}",-56549.686306,-56913.367396,-57675.888398,-57046.314033,469.282356,3
2,3.172615,0.035793,0.056189,2.707524e-03,2,30,"{'max_features': 2, 'n_estimators': 30}",-57555.415783,-58254.029193,-59137.367368,-58315.604115,647.295042,4
10,3.163225,0.006926,0.016954,8.142961e-04,8,10,"{'max_features': 8, 'n_estimators': 10}",-58727.221749,-58155.221529,-59999.439213,-58960.627497,770.775972,5
7,2.553470,0.049188,0.018617,9.403244e-04,6,10,"{'max_features': 6, 'n_estimators': 10}",-58979.517997,-59014.217516,-60364.899735,-59452.878416,645.052028,6
4,1.788431,0.016266,0.017952,8.139067e-04,4,10,"{'max_features': 4, 'n_estimators': 10}",-58963.948891,-59573.549877,-60349.639842,-59629.046203,567.065390,7
1,1.010557,0.025248,0.019947,2.154609e-03,2,10,"{'max_features': 2, 'n_estimators': 10}",-59192.265935,-60355.441277,-61590.428118,-60379.378443,979.191913,8
9,1.033615,0.009822,0.007646,2.350895e-03,8,3,"{'max_features': 8, 'n_estimators': 3}",-65417.802745,-65572.363560,-67518.556592,-66169.574299,955.959268,9
6,0.782818,0.030603,0.006649,4.701342e-04,6,3,"{'max_features': 6, 'n_estimators': 3}",-66378.290252,-66899.550412,-69040.109093,-67439.316586,1151.761123,10


In [50]:
# extra code – these few lines of code just make the DataFrame look nicer
cv_res = cv_res[[
                 "param_n_estimators","param_max_features", "split0_test_score",
                 "split1_test_score", "split2_test_score", "mean_test_score"]]
#score_cols = ["split0", "split1", "split2", "mean_test_mae"]
#cv_res.columns = ["n_estimators","max_features"] + score_cols
#cv_res[score_cols] = -cv_res[score_cols].round().astype(np.int64)

cv_res

,param_n_estimators,param_max_features,split0_test_score,split1_test_score,split2_test_score,mean_test_score
0,3,2,-66789.483447,-68613.729018,-69713.005163,-68372.072542
1,10,2,-59192.265935,-60355.441277,-61590.428118,-60379.378443
2,30,2,-57555.415783,-58254.029193,-59137.367368,-58315.604115
3,3,4,-67085.659909,-67019.918992,-68263.121490,-67456.233464
4,10,4,-58963.948891,-59573.549877,-60349.639842,-59629.046203
5,30,4,-56549.686306,-56913.367396,-57675.888398,-57046.314033
6,3,6,-66378.290252,-66899.550412,-69040.109093,-67439.316586
7,10,6,-58979.517997,-59014.217516,-60364.899735,-59452.878416
8,30,6,-56641.604309,-56607.720035,-57791.547673,-57013.624006
9,3,8,-65417.802745,-65572.363560,-67518.556592,-66169.574299


In [51]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.328137,0.024430,0.006981,2.247832e-07,2,3,"{'max_features': 2, 'n_estimators': 3}",-66789.483447,-68613.729018,-69713.005163,-68372.072542,1205.692939,12
1,1.010557,0.025248,0.019947,2.154609e-03,2,10,"{'max_features': 2, 'n_estimators': 10}",-59192.265935,-60355.441277,-61590.428118,-60379.378443,979.191913,8
2,3.172615,0.035793,0.056189,2.707524e-03,2,30,"{'max_features': 2, 'n_estimators': 30}",-57555.415783,-58254.029193,-59137.367368,-58315.604115,647.295042,4
3,0.545422,0.016288,0.006649,9.406053e-04,4,3,"{'max_features': 4, 'n_estimators': 3}",-67085.659909,-67019.918992,-68263.121490,-67456.233464,571.186883,11
4,1.788431,0.016266,0.017952,8.139067e-04,4,10,"{'max_features': 4, 'n_estimators': 10}",-58963.948891,-59573.549877,-60349.639842,-59629.046203,567.065390,7
5,5.483845,0.113311,0.054363,2.820580e-03,4,30,"{'max_features': 4, 'n_estimators': 30}",-56549.686306,-56913.367396,-57675.888398,-57046.314033,469.282356,3
6,0.782818,0.030603,0.006649,4.701342e-04,6,3,"{'max_features': 6, 'n_estimators': 3}",-66378.290252,-66899.550412,-69040.109093,-67439.316586,1151.761123,10
7,2.553470,0.049188,0.018617,9.403244e-04,6,10,"{'max_features': 6, 'n_estimators': 10}",-58979.517997,-59014.217516,-60364.899735,-59452.878416,645.052028,6
8,6.890174,0.036886,0.039930,1.581218e-04,6,30,"{'max_features': 6, 'n_estimators': 30}",-56641.604309,-56607.720035,-57791.547673,-57013.624006,550.249011,2
9,1.033615,0.009822,0.007646,2.350895e-03,8,3,"{'max_features': 8, 'n_estimators': 3}",-65417.802745,-65572.363560,-67518.556592,-66169.574299,955.959268,9


## Gridsearch mit MAPE

In [52]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = [
    { 'max_features': [2,4,6,8],
      'n_estimators': [3,10,30]}# oder ...[10,100,1000,10000]
#,
#    {'bootstrap': [False],
#     'max_features': [2, 3,4],
#     'n_estimators': [3, 10]}
]
#param_grid enthält eine Dictionary, das ausgewählte Parameter von der Klasse RandomForest zeigt.
# Diese Parameter heißen HYPER-PARAMETER
# Die Listen, die hinter den Parametern stehen , enthalten mögliche Werte der Parameter.

grid_search = GridSearchCV(RandomForestRegressor(random_state=42), 
                           param_grid, 
                           cv=3,
                           scoring='neg_mean_absolute_percentage_error',
                           n_jobs = -1)
grid_search.fit(housing_train_data, housing_train_labels)
# Das grid_search-Objekt führt ein fit für jede Parameter-kombination aus und macht auch eine Kreuz-Evaluierung(Crossval)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]}],
             scoring='neg_mean_absolute_percentage_error')

In [53]:
# Die grid_search-Instanz findet die besten Parameter
grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

In [54]:
print( "Score der besten Parameterkombination",-np.round(grid_search.best_score_ *100,2),"%")

Score der besten Parameterkombination 22.05 %


In [55]:
# Die grid_search-Instanz kann auch einen Estimator erstellen, der die besten Parameter schon enthält.
final_model=grid_search.best_estimator_
final_model

RandomForestRegressor(max_features=8, n_estimators=30, random_state=42)

## Evaluate Your System on the Test Set

In [56]:
with open('datasets/housing_test_prepared.pickle', 'rb') as f3:
    housing_test_data = pickle.load(f3)
print("Testdaten",housing_test_data)

with open('datasets/housing_test_labels.pickle', 'rb') as f3:
    housing_test_labels = pickle.load(f3)
print("Testdaten",housing_test_labels)

Testdaten [[-7.e-01 -4.e-01  3.e-01 ...  0.e+00  0.e+00  0.e+00]
 [-4.e-01  1.e-01 -8.e-02 ...  0.e+00  0.e+00  0.e+00]
 [-8.e-01 -5.e-01 -2.e-01 ...  0.e+00  0.e+00  0.e+00]
 ...
 [ 3.e-01  1.e+00  2.e+00 ...  0.e+00  0.e+00  0.e+00]
 [-3.e-02  8.e-01  2.e-03 ...  0.e+00  0.e+00  0.e+00]
 [ 5.e+00 -3.e+00 -1.e+00 ...  0.e+00  0.e+00  0.e+00]]
Testdaten 8290     113600.0
11221     61300.0
2008     331500.0
8783      84200.0
17027    152400.0
           ...   
14782    238300.0
18371     96600.0
20286    150000.0
19257     77700.0
11919    143700.0
Name: median_house_value, Length: 4128, dtype: float64


In [57]:
final_predictions = final_model.predict(housing_test_data)

from sklearn.metrics import mean_squared_error
final_rmse = np.round(mean_squared_error(housing_test_labels, final_predictions, squared=False))
print(final_rmse,"$")

55796.0 $


In [58]:
from sklearn.metrics import mean_absolute_percentage_error
final_mape = np.round(mean_absolute_percentage_error(housing_test_labels, final_predictions)*100)
print(final_mape,"%")

21.0 %


In [55]:
# Anmerkung: Der Score ist hier so schlecht, 
# weil die Testmenge offenbar eine andere STruktur hat asl die Trainingsmenge

Nur für Leute mit guten Statistik kenntnissen:
Wir können ein 95% Konfidenz-Intervall für die test-RMSE konstruieren

In [59]:
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - housing_test_labels) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))

array([53415., 58080.])

In [60]:
# extra code – shows how to compute a confidence interval for the RMSE
m = len(squared_errors)
mean = squared_errors.mean()
tscore = stats.t.ppf((1 + confidence) / 2, df=m - 1)# T-Verteilung !
tmargin = tscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - tmargin), np.sqrt(mean + tmargin)

(53414.62618182245, 58080.038037041086)

Falls der Chef das Modell gut genug findet, kann es ab jetzt im Unternehmen eingesetzt werden.